In [4]:
# 日期 取值 集合


# d_list= ['180418', '180419', '180420', '180421', '180422', '180423', '180424', '180425',\
#          '180426', '180427', '180428', '180429', '180430', '180501', '180502', '180503', \
#          '180504', '180505', '180506', '180507', '180508', '180509', '180510', '180511', \
#          '180512', '180513', '180514', '180515', '180516', '180517', '180518', '180519', \
#          '180520', '180521', '180522', '180523', '180524', '180525', '180526', '180527', \
#          '180528', '180529', '180530', '180531']

#d_list= ['180418', '180419', '180420', '180421', '180422', '180423', '180424', '180425']


d_list = ['180602', '180603', '180604', '180605', '180606', '180607', '180608', '180609', '180610',\
          '180611', '180612', '180613', '180614', '180615', '180616', '180617', '180618', '180619', '180620',\
          '180621', '180622', '180623', '180624', '180625', '180626', '180627', '180628', '180629', '180630',\
          '180701', '180702', '180703', '180704', '180705', '180706', '180707', '180708', '180709', '180710', \
          '180711', '180712', '180713', '180714', '180715']


# d_list = [ '180709', '180710','180711', '180712', '180713', '180714', '180715']

In [9]:
import pandas as pd
import time

# 替换一些命令前缀
cmd_class = ('perl','touch','sh','python','TDSQLLoader.pl','TDSQLLoaderHis.pl','TDDQCCheck.pl','TDDQCCheckHis.pl','TDDMCheck.pl')
def f_cmdline(cmd):
    if cmd in cmd_class:
        cmd = cmd.replace('TDSQLLoaderHis','TDSQLLoader')
        cmd = cmd.replace('TDDQCCheckHis','TDDQCCheck')
        return cmd
    elif cmd =='':
        return 'cmd_none'
    else:
        return 'cmd_other'
def f_s(s):
    return s[1:]
def f_d(s):
    return s[3:9]

### ajob表__处理

In [10]:
#分10份处理数据,10个日期   按v_date  划分；加入cmdline特征 
chunkSize = 100000
clums = ('order_id','task_type','job_name','job_id','odate','max_wait','confirm_flag','from_time','to_time','interval'\
,'next_time','priority','cpu_id','max_rerun','avg_runtime','status','state','delete_flag','start_time','end_time'\
,'rerun_counter','cmdline','cyclic','nodegroup','order_table','order_time','avg_start_time','cpu_time',\
 'prev_odate_rerun_counter','v_date')

cls_todo = [cl for cl in clums if cl not in ('order_id','v_date')]

for i in range(len(d_list)//10+1):
#     if i != 12 :
#         continue
    print(i)
    start = time.time()
    day_i = d_list[i*10:(i+1)*10]
    chunks = [[],[],[],[],[],[],[],[],[],[]]
    
    day_len = len(day_i)
        
    f = open(r'E:\aevt\AJOB_TMP_20180715.DAT')
    reader = pd.read_csv(f, sep='|',header=None, iterator=True)
    
    loop = True
    i=0
    while loop:
        try:
            i += 1
            if i%100 == 0:
                print(i,end=' ')
    #             break
            chunk = reader.get_chunk(chunkSize)
            chunk = chunk[[0,1,2,3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,24,25,26,27,28,29,30,31]]
            chunk.columns = clums

            chunk['order_d'] = chunk['order_time'].apply(f_d)
            
            chunk['server'] = chunk['v_date'].apply(lambda x:x[7:10])
            chunk['v_date'] = chunk['v_date'].apply(lambda x:x[1:7])
            
            chunk['cmdline'] = chunk['cmdline'].apply(lambda x: x.split(' ')[0])

            for k in range(day_len):
                chunks[k].append(chunk[chunk['order_d']==day_i[k]])

        except StopIteration:
            loop = False
            print("Iteration is stopped.")
    
    

    
    for j in range(day_len):
        df = pd.concat(chunks[j], ignore_index=True)
        
        for cl in cls_todo:
            df[cl] = df[cl].apply(f_s)
            
        df['cmdline'] = df['cmdline'].apply(f_cmdline)
        #df['order_id'] = df[['order_id','server']].apply(lambda x:x[0]+x[1][1:],axis=1)
#         df['server'] = df['v_date'].apply(lambda x:x[7:11])
#         df['v_date'] = df['v_date'].apply(lambda x:x[:7])
        
        df.to_csv(r'E:\jupyter\CTM\data_src\ajob\ajob_%s_0.txt'%(df['v_date'][0]),index=False)

    end = time.time()
    print('%0.2f S'%(end-start))

0
   100
Iteration is stopped.
206.12 S
1
   100
Iteration is stopped.
207.61 S
2
   100
Iteration is stopped.
218.17 S
3
   100
Iteration is stopped.
209.72 S
4
   100
Iteration is stopped.
179.10 S


### 作业对应资源表处理

In [4]:
#  
#分10份处理数据,10个日期

chunkSize = 100000
clums = ('order_id','resource','quant','v_date')

cls_todo = clums[1:-1]

for i in range(len(d_list)//10+1):
#     if i != 12 :
#         continue
    print(i)
    start = time.time()
    day_i = d_list[i*10:(i+1)*10]
    chunks = [[],[],[],[],[],[],[],[],[],[]]
    
    day_len = len(day_i)
        
    f = open(r'E:\aevt\ALNKI_Q_TMP_20180715.DAT')
    reader = pd.read_csv(f, sep='|',header=None, iterator=True)
    
    loop = True
    i=0
    while loop:
        try:
            i += 1
            if i%100 == 0:
                print(i,end=' ')
    #             break
            chunk = reader.get_chunk(chunkSize)
            chunk = chunk[[0,1,2,6]]
            chunk.columns = clums
            #chunk['v_date'] = chunk['v_date'].apply(f_s)     
            chunk['server'] = chunk['v_date'].apply(lambda x:x[7:10]) 
            chunk['v_date'] = chunk['v_date'].apply(lambda x:x[1:7]) 
            for k in range(day_len):
                chunks[k].append(chunk[chunk['v_date']==day_i[k]])

        except StopIteration:
            loop = False
            print("Iteration is stopped.")
    
    

    
    for j in range(day_len):
        df = pd.concat(chunks[j], ignore_index=True)
        
        for cl in cls_todo:
            df[cl] = df[cl].apply(f_s)
        df.to_csv(r'E:\jupyter\CTM\data_src\resorce\job_res_num_%s.csv'%(df['v_date'][0]),index=False)

    end = time.time()
    print('%0.2f s'%(end-start))

0
   100
   200
Iteration is stopped.
78.31 s
1
   100
   200
Iteration is stopped.
75.49 s
2
   100
   200
Iteration is stopped.
78.23 s
3
   100
   200
Iteration is stopped.
77.72 s
4
   100
   200
Iteration is stopped.
60.54 s


### 依赖关系输入表__处理

In [5]:
#condition ——I 
#分10份处理数据,10个日期

chunkSize = 100000
clums = ('order_id','condition','odate','and_or','v_date')

cls_todo = clums[1:-1]

for i in range(len(d_list)//10+1):
#     if i != 12 :
#         continue
    print(i)
    start = time.time()
    day_i = d_list[i*10:(i+1)*10]
    chunks = [[],[],[],[],[],[],[],[],[],[]]
    
    day_len = len(day_i)
        
    f = open(r'E:\aevt\ALNKI_P_TMP_20180715.DAT')
    reader = pd.read_csv(f, sep='|',header=None, iterator=True)
    
    loop = True
    i=0
    while loop:
        try:
            i += 1
            if i%100 == 0:
                print(i,end=' ')
    #             break
            chunk = reader.get_chunk(chunkSize)
            chunk = chunk[[0,1,2,3,6]]
            chunk.columns = clums
            #chunk['v_date'] = chunk['v_date'].apply(f_s)     
            chunk['server'] = chunk['v_date'].apply(lambda x:x[7:10]) 
            chunk['v_date'] = chunk['v_date'].apply(lambda x:x[1:7]) 
            for k in range(day_len):
                chunks[k].append(chunk[chunk['v_date']==day_i[k]])

        except StopIteration:
            loop = False
            print("Iteration is stopped.")
    
    for j in range(day_len):
        df = pd.concat(chunks[j], ignore_index=True)
        
        for cl in cls_todo:
            df[cl] = df[cl].apply(f_s)
            
        df['up'] = df['condition'].apply(lambda x:x.split('-TO-')[0])
        df['down'] = df['condition'].apply(lambda x:x.split('-TO-')[-1])
        del df['condition']
        df.to_csv(r'E:\jupyter\CTM\data_src\condition_i/conditin_i_%s.csv'%(df['v_date'][0]),index=False)

    end = time.time()
    print('%0.2f S'%(end-start))

0
   100
Iteration is stopped.
65.69 S
1
   100
Iteration is stopped.
61.99 S
2
   100
Iteration is stopped.
62.54 S
3
   100
Iteration is stopped.
61.71 S
4
   100
Iteration is stopped.
47.65 S


### 依赖关系输出表__处理

In [6]:
#condition ——O
#分10份处理数据,10个日期
chunkSize = 100000
clums = ('order_id','condition','odate','sign','v_date')

cls_todo = clums[1:-1]

for i in range(len(d_list)//10+1):
#     if i != 12 :
#         continue
    print(i)
    start = time.time()
    day_i = d_list[i*10:(i+1)*10]
    chunks = [[],[],[],[],[],[],[],[],[],[]]
    
    day_len = len(day_i)
        
    f = open(r'E:\aevt\ALNKO_P_TMP_20180715.DAT')
    reader = pd.read_csv(f, sep='|',header=None, iterator=True)
    
    loop = True
    i=0
    while loop:
        try:
            i += 1
            if i%100 == 0:
                print(i,end=' ')
    #             break
            chunk = reader.get_chunk(chunkSize)
            chunk = chunk[[0,1,2,3,5]]
            chunk.columns = clums
            #chunk['v_date'] = chunk['v_date'].apply(f_s）
            chunk['server'] = chunk['v_date'].apply(lambda x:x[7:10]) 
            chunk['v_date'] = chunk['v_date'].apply(lambda x:x[1:7])
            for k in range(day_len):
                chunks[k].append(chunk[chunk['v_date']==day_i[k]])

        except StopIteration:
            loop = False
            print("Iteration is stopped.")
    
    

    
    for j in range(day_len):
        df = pd.concat(chunks[j], ignore_index=True)
        
        for cl in cls_todo:
            df[cl] = df[cl].apply(f_s)
            
        df['up'] = df['condition'].apply(lambda x:x.split('-TO-')[0])
        df['down'] = df['condition'].apply(lambda x:x.split('-TO-')[-1])
        del df['condition']
        df.to_csv(r'E:\jupyter\CTM\data_src\condition_o\conditin_o_%s.csv'%(df['v_date'][0]),index=False)

    end = time.time()
    print('%0.2f S'%(end-start))

0
   100
   200
Iteration is stopped.
136.28 S
1
   100
   200
Iteration is stopped.
139.31 S
2
   100
   200
Iteration is stopped.
135.83 S
3
   100
   200
Iteration is stopped.
133.93 S
4
   100
   200
Iteration is stopped.
107.56 S


### 事件表__处理

In [7]:
##ALNKI_Q_TMP_  事件表
#分10份处理数据,10个日期

chunkSize = 100000
clums = ('evt_time','new_value','old_value','order_id','v_date')

cls_todo = clums[0:-1]

for i in range(len(d_list)//5+1):
#     if i != 12 :
#         continue
    print(i)
    start = time.time()
    day_i = d_list[i*5:(i+1)*5]
    chunks = [[],[],[],[],[],[],[],[],[],[]]
    
    day_len = len(day_i)
        
    f = open(r'E:\aevt\AEVT_TMP_20180715.DAT')
    reader = pd.read_csv(f, sep='|',header=None, iterator=True)
    
    loop = True
    i=0
    while loop:
        try:
            i += 1
            if i%100 == 0:
                print(i，end=' ')
    #             break
            chunk = reader.get_chunk(chunkSize)
            chunk = chunk[[1,3,4,5,8]]
            chunk.columns = clums
            #chunk['v_date'] = chunk['v_date'].apply(f_s)
            
            chunk['server'] = chunk['v_date'].apply(lambda x:x[7:10]) 
            chunk['v_date'] = chunk['v_date'].apply(lambda x:x[1:7]) 
            chunk['evt_time'] = chunk['evt_time'].apply(lambda x:x[:-7])
            for k in range(day_len):
                chunks[k].append(chunk[chunk['v_date']==day_i[k]])

        except StopIteration:
            loop = False
            print("Iteration is stopped.")
    
    

    
    for j in range(day_len):
        df = pd.concat(chunks[j], ignore_index=True)
        
        for cl in cls_todo:
            df[cl] = df[cl].apply(f_s)
        
        df.to_csv(r'E:\jupyter\CTM\data_src\evet\evet_%s.csv'%(df['v_date'][0]),index=False)

    end = time.time() 
    print('%0.2f mintue'%((end-start)/60))

0
   100
   200
   300
   400
   500
   600
   700
   800
   900
   1000
   1100
   1200
   1300
   1400
   1500
   1600
Iteration is stopped.
21.19 mintue
1
   100
   200
   300
   400
   500
   600
   700
   800
   900
   1000
   1100
   1200
   1300
   1400
   1500
   1600
Iteration is stopped.
17.50 mintue
2
   100
   200
   300
   400
   500
   600
   700
   800
   900
   1000
   1100
   1200
   1300
   1400
   1500
   1600
Iteration is stopped.
13.82 mintue
3
   100
   200
   300
   400
   500
   600
   700
   800
   900
   1000
   1100
   1200
   1300
   1400
   1500
   1600
Iteration is stopped.
10.45 mintue
4
   100
   200
   300
   400
   500
   600
   700
   800
   900
   1000
   1100
   1200
   1300
   1400
   1500
   1600
Iteration is stopped.
10.14 mintue
5
   100
   200
   300
   400
   500
   600
   700
   800
   900
   1000
   1100
   1200
   1300
   1400
   1500
   1600
Iteration is stopped.
10.71 mintue
6
   100
   200
   300
   400
   500
   600
   700
   800
   900